# k.p example of using the Lowdin perturbation

This example requires [semicon](https://gitlab.kwant-project.org/semicon/semicon) to be installed.

It should be as easy as 
```
pip install git+https://gitlab.kwant-project.org/semicon/semicon.git
```

In [ ]:
try:
    import semicon
except ImportError:
    print("Semicon should be installed to run this notebook.")

In [ ]:
import kwant
import scipy
import sympy
import semicon


print("semicon version:", semicon.__version__)
print("kwant version:", kwant.__version__)
print("scipy version:", scipy.__version__)
print("sympy version", sympy.__version__)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=kwant.KwantDeprecationWarning)

In [ ]:
import kwant
import semicon

import numpy as np
import scipy.linalg as la
import scipy.sparse
import scipy.sparse.linalg as sla

import sympy
sympy.init_printing(print_builtin=False)

import matplotlib.pyplot as plt
%matplotlib inline

import codes.old_lowdin as olo
import codes.higher_order_lowdin as nlo

In [ ]:
%load_ext line_profiler

# Prepare k.p model and solve it exactly

In [ ]:
widths = [5, 12.5, 5, 5]
gamma_0 = 1.0

grid_spacing = 0.5
shape = lambda site: 0 - grid_spacing / 2 < site.pos[0] < sum(widths)

model = semicon.models.ZincBlende(
    components=('foreman',),
    parameter_coords='z', 
    default_databank='lawaetz'
)

kpham = model.hamiltonian

AlSb = model.parameters('AlSb', valence_band_offset=.18).renormalize(new_gamma_0=gamma_0)
InAs = model.parameters('InAs').renormalize(new_gamma_0=gamma_0)
GaSb = model.parameters('GaSb', valence_band_offset=.56).renormalize(new_gamma_0=gamma_0)

params, walls = semicon.misc.two_deg(
    parameters = [AlSb, InAs, GaSb, AlSb],
    widths = widths,
    grid_spacing=grid_spacing,
    extra_constants=semicon.parameters.constants,
)

In [ ]:
def calc_energy(k):
    p = {'k_x': k, 'k_y': 0}
    mat = syst.hamiltonian_submatrix(params={**p, **params})
    return la.eigvalsh(mat)


template = kwant.continuum.discretize(str(kpham), coords='z', 
                                      grid_spacing=grid_spacing)

syst = kwant.Builder()
syst.fill(template, shape, (0, ))
syst = syst.finalized()

N = len(syst.sites)
momenta = np.linspace(-.3, .3, 81)
energies_exact = np.array([calc_energy(k) for k in momenta])

In [ ]:
ylims = (500-5, 530+5)
xlims = (-.3, .3)

window = (.500, .530)

plt.plot(momenta, 1000 * energies_exact, 'k-');
plt.plot(xlims, 1000 * np.array([window, window]))
plt.ylim(*ylims)
plt.xlim(*xlims)

# prepare perturbation basis

In [ ]:
def find_indices(ev, lower, upper):
    return [i for (i, e) in enumerate(ev) if lower < e < upper]

In [ ]:
# for simplicity lets import everything now
from codes.lowdin import *

gens = sympy.symbols(['k_x', 'k_y'])
H0, H1 = prepare_hamiltonian(
    kpham, gens, 'z', grid_spacing, shape, (0,)
)

## Full diagonalisation: we know everything

In [ ]:
mat0 = H0.hamiltonian_submatrix(params=params, sparse=True)
mat1 = {k: v.hamiltonian_submatrix(params=params) 
        for k, v in H1.items()}

ev, evec = la.eigh(mat0.A)
indices = find_indices(ev, *window) 

## Old lowdin at higher order

In [ ]:
from importlib import reload
olo = reload(olo)

In [ ]:
%%time
order=6
gens = sympy.symbols('k_x k_y')
inter_keys = olo.get_maximum_powers(gens, order)

model = olo.get_effective_model(ev, evec, indices, mat1, inter_keys)

ham = sympy.expand(model.tosympy())

In [ ]:
model_old = model
ham

In [ ]:
# %lprun -f  olo.get_effective_model olo.get_effective_model(ev, evec, indices, mat1, inter_keys)

## KPM lowdin at higher order

In [ ]:
from importlib import reload
nlo = reload(nlo)

In [ ]:
indices

In [ ]:
%%time
order=6
kpm_params = dict(num_moments=1000)
gens = sympy.symbols('k_x k_y')
inter_keys = nlo.get_maximum_powers(gens, order)

# Getting sparse vectors results in different basis, result looks different
# energies, evec_A = sla.eigsh(mat0, sigma=.520, k=4)
# evec_A, _ = la.qr(evec_A[:, np.argsort(energies)], mode='economic')
evec_A = evec[:, indices]
# B states to include exactly, 3 options: some, all, none
# evec_B = evec[:, np.arange(300, 328)]
# evec_B = np.hstack([evec[:, np.arange(332, 440)], evec[:, np.arange(0, 328)]])
evec_B = None

model = nlo.get_effective_model(mat0, mat1, evec_A, evec_B,
                                interesting_keys=inter_keys, order=order, kpm_params=kpm_params)

ham = sympy.expand(model.tosympy())

In [ ]:
# %lprun -f  nlo.get_effective_model nlo.get_effective_model(mat0, mat1, evec_A, evec_B, inter_keys, order=6, kpm_params=kpm_params)

In [ ]:
# %lprun -f  nlo.divide_by_energies nlo.get_effective_model(mat0, mat1, evec_A, inter_keys, order=4, kpm_params=kpm_params)

In [ ]:
inter_keys

In [ ]:
model_new = model
ham

In [ ]:
for key in model_old.keys():
    assert nlo.allclose(model_old[key], model_new[key], rtol=1e-1), (key, model_old[key] - model_new[key])

## Old lowdin at higher order

In [ ]:
from importlib import reload
olo = reload(olo)

In [ ]:
# Simple model
ev2 = np.array([1., 5.])
mat02 = np.diag(ev2)
mat12 = {sympy.Symbol('x'): np.array([[0, 1.], [1., 0.5]])}
evec2 = np.eye(2)
indices2 = [0]

In [ ]:
gens2 = [sympy.symbols('x')]
inter_keys2 = olo.get_maximum_powers(gens2, 8)

model = olo.get_effective_model(ev2, evec2, indices2, mat12, inter_keys2)

ham = sympy.expand(model.tosympy())

In [ ]:
inter_keys2

In [ ]:
model_old2 = model
ham

## KPM lowdin at higher order

In [ ]:
import codes.higher_order_lowdin as nlo

In [ ]:
from importlib import reload
nlo = reload(nlo)

In [ ]:
kpm_params = dict(num_moments=1000)

evec_A = evec2[:, indices2]
evec_B = evec2[:, np.array([1])]
# evec_B = None

model = nlo.get_effective_model(mat02, mat12, evec_A, evec_B, interesting_keys=inter_keys2, order=8, kpm_params=kpm_params)

ham = sympy.expand(model.tosympy())

In [ ]:
inter_keys2

In [ ]:
model_new2 = model
ham

In [ ]:
for key in model_old2.keys():
    assert nlo.allclose(model_old2[key], model_new2[key], atol=1e-6), (key, model_old2[key] - model_new2[key])